# Analysis of the Pattern Concept Lattice of Code Templates




In [1]:
import json
import networkx as nx
import ipywidgets as widgets
from IPython.display import display

## Loading the lattice
Loading the content of the json file representing the concepts and their relations.

In [2]:
filepath = './template-lattice_small.json'
with open(filepath, 'r') as jsonfile:
    lattice = json.loads(jsonfile.read())
jsonfile.close()

## Create a digraph
Put the content of the json in a networkx digraph to perform some graph operations.  
The digraph is transitively closed.

In [3]:
G = nx.DiGraph()
for edge in lattice['edges']:
    if edge[0] != edge[1]:
        G.add_edge(edge[0], edge[1])
        
print('Number of nodes: ' + str(G.number_of_nodes()))
print('Number of edges (TC): ' + str(G.number_of_edges()))

Number of nodes: 77833
Number of edges (TC): 2071600


## Compute the transitive reduction
We want the edges to show only the direct neighbors of a node.
(Can take a few seconds)

In [4]:
Gtr = nx.transitive_reduction(G)
print('Number of edges (TR): ' + str(Gtr.number_of_edges()))

Number of edges (TR): 197356


## Finding the top concept
Retrieve the node that has no parent. Only one node should verifies that.  
We need this top-concept to start the exploration from the most general node.

In [5]:
top_concepts = []
for id_node in Gtr.nodes:
    if len(list(Gtr.successors(id_node))) == 0:
        top_concepts.append(id_node)
        
assert(len(top_concepts) == 1)
top_concept = top_concepts[0]
print('ID of the top-concept: ' + str(top_concept))

ID of the top-concept: 17167


## Navigation interface

In [6]:
current_concept = top_concept

def on_button_clicked(b):
    global current_concept
    current_concept = int(b.tooltip)
    global box_concept
    box_concept.value = str(lattice['concepts'][current_concept-1]['pattern']) + ' - ' + str(len(lattice['concepts'][current_concept-1]['conforming_fragments']))

def create_template_buttons(change=None):
    sub_concepts = list(Gtr.predecessors(current_concept))
    sup_concepts = list(Gtr.successors(current_concept))
    
    # buttons for sub-concepts
    buttons_sub = [widgets.Button(
                description=lattice['concepts'][c-1]['pattern'] + ' - ' + str(len(lattice['concepts'][c-1]['conforming_fragments'])),
                disabled=False,
                button_style='', 
                tooltip=str(c),
                layout=widgets.Layout(width='98%')
                ) for c in sub_concepts]
    # sorted by number of conforming fragments
    buttons_sub = sorted(buttons_sub, key=lambda x:len(lattice['concepts'][int(x.tooltip)-1]['conforming_fragments']), reverse=True)
    # clickable
    for button in buttons_sub:
        button.on_click(on_button_clicked) 
   
    # buttons for sup-concept
    buttons_sup = [widgets.Button(
                description=str(lattice['concepts'][c-1]['pattern']) + ' - ' + str(len(lattice['concepts'][c-1]['conforming_fragments'])),
                disabled=False,
                button_style='', 
                tooltip=str(c),
                layout=widgets.Layout(width='98%')
                ) for c in sup_concepts]
    # sorted by number of conforming fragments
    buttons_sup = sorted(buttons_sup, key=lambda x:len(lattice['concepts'][int(x.tooltip)-1]['conforming_fragments']), reverse=True)
    # clickable
    for button in buttons_sup:
        button.on_click(on_button_clicked) 
        
    return buttons_sub, buttons_sup


# current concept
box_concept = widgets.Text(
    value=str(lattice['concepts'][current_concept-1]['pattern']) + ' - ' + str(len(lattice['concepts'][current_concept-1]['conforming_fragments'])),
    placeholder='',
    description='',
    disabled=False
)

# tabs
tabs = widgets.Tab()
tabs.set_title(0,'Specialization')
tabs.set_title(1,'Generalization')
buttons_sub, buttons_sup = create_template_buttons()
box_buttons_sub = widgets.GridBox(buttons_sub, layout=widgets.Layout(grid_template_columns="repeat(3, 33%)"))
box_buttons_sup = widgets.GridBox(buttons_sup, layout=widgets.Layout(grid_template_columns="repeat(3, 33%)"))
tabs.children = [box_buttons_sub, box_buttons_sup]
     
    
# switching current concept
def cc_change(change):
    buttons_sub, buttons_sup = create_template_buttons()
    box_buttons_sub.children = buttons_sub
    box_buttons_sup.children = buttons_sup

    
box_concept.observe(cc_change, names='value')    
display(box_concept)
display(tabs)

Text(value=' - 25162', placeholder='')